## What is Transfer Learning?

**Transfer Learning** is a deep learning technique where a pre-trained model (trained on a large dataset) is fine-tuned or adapted to a new task, usually with a smaller dataset. This significantly reduces training time, computational cost, and data requirements while improving performance.

When we apply transfer learning, we typically freeze the early layers of a pre-trained model and only fine-tune the later layers or add a custom classifier.

1. Pre-Trained Feature Extractor:

The lower layers of a CNN (for vision tasks) or Transformer (for NLP tasks) extract generic low-level features such as edges, textures, or general patterns.
These features are universal across many datasets and do not need to be retrained.

2. Freezing Lower Layers:

Since early layers capture general features, they remain frozen to retain learned representations.
Freezing prevents unnecessary weight updates, allowing the model to leverage prior knowledge.

3. Fine-Tuning Higher Layers:

The deeper layers of the model are task-specific and need fine-tuning to adapt to new data.
These layers learn complex, high-level features such as object parts, shapes, or specific language patterns.

4. Adding a Custom Classifier:

The final fully connected layers are often replaced with a new classifier tailored for the target task.
This part of the model is trained from scratch while leveraging extracted features.

**NOTE!** Transfer learning helps you get reliable results with less data.

Let's firstly check the processor info.

In [26]:
# GPU or CPU
!nvidia-smi

Sat Mar 22 20:13:09 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   34C    P8              9W /   70W |       2MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

**Paho0o!**
Apprently, we're not on GPU for now.

No worries, we will take care of it as soon as GPU is needed.

Now, let us install the necessary tools and packages.

In [27]:
!pip install -U tensorflow_hub
# !pip install tensorflow==2.15.0 tensorflow-hub keras==2.15.0

### Data Preparation

It is clear that we will need a load of data to implement our model. So let's download one provided by [Daniel Bourke](https://www.mrdbourke.com/) (Food Classes). By the way, the whole document is inspired by his work.

In [28]:
import zipfile

# Download data
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip

# Unzip the file
zip_ref = zipfile.ZipFile("10_food_classes_10_percent.zip", "r")
zip_ref.extractall()
zip_ref.close()

--2025-03-22 20:13:12--  https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.197.207, 74.125.135.207, 74.125.195.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.197.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 168546183 (161M) [application/zip]
Saving to: ‘10_food_classes_10_percent.zip.3’

10_food_classes_10_ 100%[===================>] 160.74M   182MB/s    in 0.9s    

2025-03-22 20:13:13 (182 MB/s) - ‘10_food_classes_10_percent.zip.3’ saved [168546183/168546183]



In [29]:
import os

# explore the data
for dirpath, dirnames, filenames in os.walk("10_food_classes_10_percent"):
  print(f"There are {len(dirnames)} directories and {len(filenames)} images in '{dirpath}'.")

There are 2 directories and 0 images in '10_food_classes_10_percent'.
There are 10 directories and 0 images in '10_food_classes_10_percent/train'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/sushi'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/hamburger'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/ice_cream'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/grilled_salmon'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/chicken_wings'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/fried_rice'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/chicken_curry'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/pizza'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/steak'.
There are 0 directories and 75 images in '10_food_classes_10_percent

Our data is composed of 10 classes, that is, 10 different types of food.

In order to be able to load our images we will have to `ImageDataGenerator()`. `ImageDataGenerator()` is a class in Keras (part of TensorFlow) that allows real-time data augmentation and efficient loading of images during model training. It is mainly used to:

- **Augment images** (apply transformations like rotation, flipping, scaling, etc.).
- **Normalize pixel values** (rescale intensity to a certain range).
- **Load images in batches** (useful for handling large datasets without loading everything into memory at once).

**How Does It Work?** When you create an ImageDataGenerator object, you specify the transformations you want to apply to your images. Then, you generate augmented batches from the dataset using `.flow()` or `.flow_from_directory()`.

In [30]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
# import tensorflow as tf

IMAGE_SHAPE = (224, 224)
BATCH_SIZE = 32

train_dir = "10_food_classes_10_percent/train/"
test_dir = "10_food_classes_10_percent/test/"

train_datagen = ImageDataGenerator(rescale=1/255.)
test_datagen = ImageDataGenerator(rescale=1/255.)

train_data = train_datagen.flow_from_directory(train_dir,
                                               target_size=IMAGE_SHAPE,
                                               batch_size=BATCH_SIZE,
                                               class_mode="categorical")

test_data = test_datagen.flow_from_directory(test_dir,
                                             target_size=IMAGE_SHAPE,
                                             batch_size=BATCH_SIZE,
                                             class_mode="categorical")

Found 750 images belonging to 10 classes.
Found 2500 images belonging to 10 classes.


### TensorBoard Callback
**TensorBoard** is a visualization tool that helps monitor and debug deep learning models by tracking metrics like loss, accuracy, gradients, histograms, and more during training. A **TensorBoard Callback** is a function in deep learning frameworks (like TensorFlow/Keras) that automatically logs this data while training a model.

In [31]:
# import tensorflow as tf
import datetime

In [32]:
# define a function to create tensorboard callback
# log name pattern: [dir_name]/[experiment_name]/[current_timestamp]
def create_tensorboard_callback(dir_name, experiment_name):
  log_dir = dir_name + "/" + experiment_name + "/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
  tensorboard_callback = tf.keras.callbacks.TensorBoard(
      log_dir=log_dir
  )
  print(f"Saving TensorBoard log files to: {log_dir}")
  return tensorboard_callback

### Creating Models using TensorFlow Hub

**TensorFlow Hub** is a repository of pre-trained machine learning models (e.g., ResNetV2, EfficientNet, etc.) that can be reused for various tasks like image classification, text embedding, object detection, and more. It provides a way to easily download, fine-tune, or use models without training from scratch.

#### Comparison with Other Model Hubs:

| Feature        | TensorFlow Hub         | Hugging Face Model Hub | PyTorch Hub  |
|:---------------|:------------------------|:------------------------|:-------------|
| **Library**   | TensorFlow/Keras        | Mainly PyTorch (some TensorFlow) | PyTorch |
| **Models**    | Vision, NLP, Generative, Speech | NLP, Vision, Multimodal | Vision, NLP |
| **Fine-tuning** | Supported              | Highly supported       | Supported |
| **Ease of Use** | Very easy              | Very easy              | Easy |


### Different Approaches for Transfer Learning

Transfer learning is a technique where a pre-trained model is adapted to a new task. Depending on how much of the pre-trained model is reused or fine-tuned, transfer learning can be implemented in different ways:

1. **Feature Extraction (Frozen Pre-Trained Model)**

Implementation:
- Use a pre-trained model as a fixed feature extractor.
- Freeze the convolutional (or encoder) layers so they retain learned features.
- Replace the fully connected (FC) layers with a new classifier suited for the new task.
- Train only the new classifier layers.

2. **Fine-Tuning (Partial Transfer Learning)
Implementation:**

- Use a pre-trained model and fine-tune some of its layers.
- Freeze the initial layers (low-level features) but allow later layers to update.
- Fine-tune with a lower learning rate to prevent catastrophic forgetting.

In [33]:
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers
import gc

In [34]:
# Resnet 50 V2 feature vector
resnet_url = "https://tfhub.dev/google/imagenet/resnet_v2_50/feature_vector/4"

# EfficientNetB0 feature vector (version 1)
efficientnet_url = "https://tfhub.dev/tensorflow/efficientnet/b0/feature-vector/1"

# EfficientNetB0 feature vector (version 2 NEW)
# efficientnet_url = "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet1k_b0/feature_vector/2"

### Create Model

We can create the models separately or define a function for it saving a good deal of unnecessary code.

**NOTE!** Be careful that the code may crash if you don't wrap the hub layer into lambda function, even taking the functional approach! In case of facing a crash, it is necessary to wrap if `tf.keras.Sequential()` is implemented.

It might raise an error similar to the one below:
```
ValueError: Only instances of `keras.Layer` can be added to a Sequential model. Received: <tensorflow_hub.keras_layer.KerasLayer object at 0x78180cf9a7d0> (of type <class 'tensorflow_hub.keras_layer.KerasLayer'>)
```

In [35]:
# # define a function to create model
def create_model(model_url, num_classes=10):
  """
  Takes a TensorFlow Hub URL and creates a Keras Sequential model
  Args:
    model_url (str): A TensorFlow Hub feature extraction URL
    num_classes (int): Number of output neurons in the output layer
    equal to number of target classes, default 10
  Returns:
    An uncompiled Keras Sequential model
  """

  # Download the pretrained model and store it as keras layer
  feature_extractor_layer = hub.KerasLayer(model_url,
                                           trainable=False, # freeze the underlying patterns
                                           name='feature_extraction_layer',
                                           input_shape=IMAGE_SHAPE+(3,))

  # JUST IN CASE!
  # hub_layer_wrapper = tf.keras.layers.Lambda(lambda x: feature_extractor_layer(x))

  # Create the model
  model = tf.keras.Sequential([
    # hub_layer_wrapper,
    feature_extractor_layer,
    layers.Dense(num_classes, activation='softmax', name='output_layer')
  ])

  return model

In [36]:
# Create model
resnet50_url = "https://www.kaggle.com/models/tensorflow/resnet-50/TensorFlow2/feature-vector/1"
resnet_model = create_model(resnet50_url, num_classes=train_data.num_classes)

The model is created now; however, there will be a few more step afterwards. Let's carry on and see what they are...

In [37]:
# Compile
resnet_model.compile(loss='categorical_crossentropy',
                     optimizer=tf.keras.optimizers.Adam(),
                     metrics=['accuracy'])

In [ ]:
# fit the model
resnet_history = resnet_model.fit(train_data,
                                  epochs=5,
                                  steps_per_epoch=len(train_data),
                                  validation_data=test_data,
                                  validation_steps=len(test_data),
                                  batch_size=16,
                                  callbacks=[create_tensorboard_callback(dir_name="tensorflow_hub_dir",
                                                                         experiment_name="tensorflow_exp_resnet50V2")])

Saving TensorBoard log files to: tensorflow_hub_dir/tensorflow_exp_resnet50V2/20250322-201324
Epoch 1/5
24/24 [==============================] - 510s 22s/step - loss: 1.9350 - accuracy: 0.3853 - val_loss: 1.0479 - val_accuracy: 0.6888
Epoch 2/5
24/24 [==============================] - ETA: 0s - loss: 0.8523 - accuracy: 0.7467